In [1]:
pip install huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install wandb

Note: you may need to restart the kernel to use updated packages.


In [4]:
from huggingface_hub import login


hf_token = ""
login(hf_token)

In [5]:
import wandb

wb_token = ""

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B on PAssFinder', 
    job_type="training", 
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: chishti055 (chishti055-north-south-university). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 1.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 86.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 43.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:010

In [7]:
from unsloth import FastLanguageModel

max_seq_length = 2048 
dtype = None 
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token, 
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [8]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [9]:
EOS_TOKEN = tokenizer.eos_token

In [10]:
from datasets import load_dataset
dataset = load_dataset("/kaggle/input/med-data", split = "train[0:500]",trust_remote_code=True)

Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  
    bias="none",  
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  
    loftq_config=None,
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

def no_formatting(data):
    return [str(data)]


def preprocess_function(examples):
    concatenated_text = [
        f"Brand: {brand} ({generic}) | Strength: {strength} | Dosage Form: {form} | "
        f"Manufacturer: {manufacturer} | Mode of Action: {moa} | Dosage: {dosage} | "
        f"Contraindications: {contraindications} | Side Effects: {side_effects} | "
        f"Precautions: {precautions} | Overdose Effects: {overdose} | Drug Classes: {classes}"
        for brand, generic, strength, form, manufacturer, moa, dosage, 
            contraindications, side_effects, precautions, overdose, classes in zip(
                examples["brand_name"],
                examples["generic_name"],
                examples["strength"],
                examples["dosage_form"],
                examples["manufacturer"],
                examples["mode_of_action"],
                examples["dosage"],
                examples["contraindications"],
                examples["side_effects"],
                examples["precautions"],
                examples["overdose_effects"],
                examples["drug_classes"]
        )
    ]
    return {"text": concatenated_text}

dataset = dataset.map(preprocess_function, batched=True)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",  # Now using the concatenated text field
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    formatting_func=no_formatting,  # Add this line to use no formatting
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs=1,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)



Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/450 [00:00<?, ? examples/s]

In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 450 | Num Epochs = 3 | Total steps = 60
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.881500
20,1.463700
30,1.331600
40,1.135800
50,1.003400
60,0.932100


In [19]:
model.save_pretrained("MedExpertLLM")
tokenizer.save_pretrained("MedExpertLLM")

('MedExpertLLM/tokenizer_config.json',
 'MedExpertLLM/special_tokens_map.json',
 'MedExpertLLM/tokenizer.json')

In [46]:
prompt_style = """ 
Write a response that appropriately completes the request and just give accurate response.

### Instruction:
You are an expert with advanced knowledge in suggesting medicine based on symptoms , diagnostics, and provide with correct solution with explanation . Please give the exact answer and suggest the medicines user gives us .. 
. 

### Question:
{}

### Response:
<think>{}"""

In [21]:
question = """ what is the generic name for 3 Bion Tablet ?
"""


FastLanguageModel.for_inference(model) 
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Okay, so I need to figure out the generic name for 3 Bion Tablet. First, I should understand what the product is. The name "3 Bion Tablet" sounds like a medication that's available in tablet form. The "3" might stand for the number of active ingredients or dosage form. "Bion" could be a brand name or a specific formulation. 

I should consider that sometimes brand names and generic names are different. The generic name is the chemical name of the active ingredient, while the brand name is the product name given by the manufacturer. So, I need to find the chemical name of the active ingredient in 3 Bion Tablet.

I should look up "3 Bion Tablet" online to see if there's a generic name associated with it. If I search for "3 Bion Tablet," I might find the manufacturer's website or other reliable sources that provide the active ingredient. Alternatively, I could search for "3 Bion Tablet active ingredient" or "3 Bion Tablet composition" to find the chemical name.

Suppose after sea

In [22]:
question = """ what are the medicnes for the asthmatic patients ?
"""


FastLanguageModel.for_inference(model) 
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Okay, I need to figure out the best medicines for asthmatic patients. First, I should understand what asthma is. Asthma is a chronic respiratory disease where the airways become inflamed and narrow, making it hard to breathe. There are different types of asthma—intermittent, persistent, and severe. Medicines vary depending on the type and severity of asthma.

I remember that short-acting beta-agonists like albuterol are commonly used to relieve symptoms during an attack. They work quickly by relaxing the muscles around the airways. Long-acting beta-agonists like formoterol are used to prevent symptoms over time. Inhaled corticosteroids like beclometasone are used to reduce inflammation in the airways. Some inhalers combine a corticosteroid with a long-acting beta-agonist, like Advair or Seretide. Leukotriene modifiers like montelukast help reduce the inflammation response.

There are also medications to reduce the immune response, like omalizumab, which is used in severe asthm

In [23]:
question = """ which medicines contain Cefixime Trihydrate ?
"""


FastLanguageModel.for_inference(model) 
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Alright, so I need to figure out which medicines contain Cefixime Trihydrate. First, I should understand what Cefixime Trihydrate is. I know that Cefixime is an antibiotic used to treat infections caused by bacteria. It belongs to a class called third-generation cephalosporins. These are broad-spectrum antibiotics that can treat a variety of bacterial infections, including infections of the ear, nose, throat, skin, and lower respiratory tract.

Cefixime is usually taken orally, in the form of a tablet or liquid suspension. It works by inhibiting bacterial cell wall synthesis. The bacteria that are susceptible to Cefixime will be killed or stopped from multiplying. This makes it effective against a wide range of bacterial infections caused by both Gram-positive and Gram-negative bacteria.

Cefixime is commonly used to treat infections such as bronchitis, pneumonia, sinusitis, otitis media, and gonorrhoea. It is also used in combination with other antibiotics to treat typhoid fe

In [49]:
question = """ which local company produces 3-C Powder for Suspension ?
"""


FastLanguageModel.for_inference(model) 
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Okay, so the user is asking which local company produces 3-C Powder for Suspension. I need to provide an accurate and helpful response. First, I should verify the information to ensure it's up-to-date. I recall that 3-C Powder is an antiviral medication used for the treatment of influenza (flu). It's also known as oseltamivir phosphate. I should check if there are any local manufacturers of this medication.

Upon checking, I find that several pharmaceutical companies in Bangladesh produce oseltamivir phosphate under different brand names. One of the prominent companies is ACME Laboratories Ltd. They manufacture this medication under the brand name "ACME-C Powder for Suspension." This is a reliable source of this antiviral medication in Bangladesh.

I should also consider the dosage form and strength of this medication. ACME-C is available as a powder for suspension and is used for oral administration. The recommended dosage for adults and children over 12 years is 75 mg twice 

Levofloxacin Hemihydrate

In [34]:
question = """ which medicines contain Levofloxacin Hemihydrate ?
"""


FastLanguageModel.for_inference(model) 
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Okay, so I need to figure out which medicines contain Levofloxacin Hemihydrate. First, I should understand what Levofloxacin Hemihydrate is. I know that Levofloxacin is an antibiotic used to treat infections caused by bacteria. It's a third-generation fluoroquinolone antibiotic, similar to other antibiotics like Ciprofloxacin and Ofloxacin. The "Hemihydrate" part probably refers to the form of the drug, meaning it's in a hydrated form, but I'm not entirely sure about the difference between that and other forms like Levofloxacin Anhydrous or Levofloxacin Tablet.

Next, I should think about brand names of Levofloxacin. I recall that Levofloxacin is sold under several brand names in different countries. In India, for example, it's sold as Levofloxacin Tablet by companies like ACME Laboratories Ltd. and ACME Laboratories Ltd. (Dangal Division). In the United States, it's sold as Levofloxacin Tablet by Janssen Pharmaceuticals Inc., and also as a intravenous infusion by the same com

Paracetamol

In [35]:
question = """ which company produces A-One Plus Tablet ?
"""


FastLanguageModel.for_inference(model) 
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Okay, so I need to figure out which company produces A-One Plus Tablet. First, I'll recall if I know any information about A-One Plus. It sounds familiar, but I'm not exactly sure who manufactures it. I think it's a common medication, so maybe it's produced by a well-known pharmaceutical company. Let me think of some companies I know produce tablets. There's Square Pharmaceuticals, Ambee Pharmaceuticals, ACI Limited, and others. I remember that Square Pharmaceuticals is involved in producing a lot of generic drugs. A-One Plus sounds like a combination of Paracetamol and Caffeine. I think Paracetamol is commonly used for headaches and pain relief. Caffeine is usually included to help with headaches and to make sure that Paracetamol works better. So, A-One Plus is likely a pain relieving tablet. Now, considering the companies that produce Paracetamol tablets, I believe Square Pharmaceuticals is one of them. I think they have a range of products, including pain relief medications

In [36]:
question = """ which company produces A-One Plus Tablet ?
"""


FastLanguageModel.for_inference(model) 
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Alright, so the user is asking about a company that produces A-One Plus Tablet. Let me break this down. First, I need to recall if I have any information about A-One Plus. It sounds familiar; I think it's a common medication. I remember that A-One Plus is used for treating certain gastrointestinal issues, maybe diarrhea or dysentery. Now, the user wants to know the manufacturer of this tablet. 

I should check my knowledge base. I recall that A-One Plus is manufactured by a well-known pharmaceutical company. I think it's a company based in Bangladesh. Was it Beximco Pharmaceuticals? That rings a bell. I should verify this to be sure. 

Let me think about the other companies. There's Square Pharmaceuticals too, but I'm pretty sure Beximco is the one. A-One Plus is a combination of two active ingredients, Loperamide and Simethicone. Loperamide is an antidiarrhoeal medication that reduces the rate of intestinal absorption of water and electrolytes. Simethicone is used to relieve 

In [45]:
question = """ which company produces Ace Pediatric Drops mainly local brand ?
"""


FastLanguageModel.for_inference(model) 
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1000,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Okay, so I need to figure out which company produces Ace Pediatric Drops as a local brand. First, I should probably understand what Ace Pediatric Drops is used for. From what I remember, pediatric drops are often used for cough, cold, and fever. So Ace Pediatric Drops is likely an oral suspension or liquid medicine used to treat these symptoms in children.

Now, the question is about the company that produces it as a local brand. Local brand usually refers to a generic version of a brand product. So, if Ace Pediatric Drops is a brand product, the local version would be produced by a different company. I should check the manufacturer of Ace Pediatric Drops. 

I think Ace Pharmaceuticals is the manufacturer of Ace Pediatric Drops. They are a pharmaceutical company based in Bangladesh. They produce various medications, including pediatric drops for cough and cold. Since the user is asking about the local brand, I need to find out which local company produces a generic version of 

In [47]:
question = """ which company produces Ace Pediatric Drops mainly local brand ?
"""


FastLanguageModel.for_inference(model) 
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1000,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Okay, so I need to figure out which company produces Ace Pediatric Drops, specifically focusing on local brands. First, I should understand what Ace Pediatric Drops are. I don't recall hearing about this specific product before, so I'll start by researching it.

I'll search online for "Ace Pediatric Drops" to see if I can find any information about it. Let me check a few search engines to see if there are any reliable sources. Hmm, the first few search results don't provide much information. Maybe it's a newer product or not widely known.

Next, I'll look for any official websites or press releases from pharmaceutical companies that might produce such a product. Let's see... After checking a few companies, I notice that there's no direct mention of Ace Pediatric Drops. This makes me think that it might be a generic or local brand product rather than a well-known international brand.

Since it's a pediatric drops product, it's likely intended for children. The term "Pediatric D

In [48]:
question = """ I have fever which medicines should I take  ?
"""


FastLanguageModel.for_inference(model) 
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1000,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Okay, so I have a fever. I'm not exactly sure what's causing it, but I need to get rid of it. I remember that fever is a symptom of many illnesses, like colds, flu, or even something more serious like malaria. But I'm not sure if I have any of those. I think I should try to bring my temperature down because it's making me feel uncomfortable.

I remember my mom used to give me Paracetamol when I had a fever as a kid. It helped reduce the pain and made me feel a bit cooler. Paracetamol is safe, right? It doesn't have too many side effects. But I'm not sure if I should take it now. Maybe it's too late? Wait, no, Paracetamol is available over the counter. I think it's okay to take it unless I have liver problems. I don't have any liver issues, so I guess it's safe.

I also remember hearing about Ibuprofen. My friend's mom uses it sometimes. It's stronger than Paracetamol, I think. But I don't know if I should take Ibuprofen. I don't have any problems with my stomach, so maybe it's

In [50]:
print(type(response))

<class 'list'>
